In [ ]:
from pathlib import Path
from time import sleep

import requests
from lumigator_schemas.datasets import DatasetFormat
from lumigator_schemas.workflows import WorkflowCreateRequest, WorkflowStatus
from lumigator_sdk.lumigator import LumigatorClient
from lumigator_sdk.strict_schemas import ExperimentIdCreate


def wait_for_workflow_complete(lumi_client_int: LumigatorClient, workflow_id: str):
    """Wait for a workflow to complete."""
    workflow_details = lumi_client_int.workflows.get_workflow(workflow_id)
    while workflow_details.status not in [WorkflowStatus.SUCCEEDED, WorkflowStatus.FAILED]:
        sleep(5)
        workflow_details = lumi_client_int.workflows.get_workflow(workflow_id)
    return workflow_details


LUMI_HOST = "localhost:8000"
with Path.open("../lumigator/sample_data/dialogsum_exc.csv") as file:
    dialog_data = file.read()

In [ ]:
lumi_client_int = LumigatorClient(api_host=LUMI_HOST)
dataset_response = lumi_client_int.datasets.create_dataset(
    dataset=dialog_data, format=DatasetFormat.JOB
)
assert dataset_response is not None
dataset_id = dataset_response.id

In [ ]:
# Create an experiment
request = ExperimentIdCreate(
    name="test_create_exp_workflow_check_results",
    description="Test for an experiment with associated workflows",
)
experiment_response = lumi_client_int.experiments.create_experiment(request)
assert experiment_response is not None
experiment_id = experiment_response.id

In [ ]:
# Create a workflow for the experiment
request = WorkflowCreateRequest(
    name="Workflow_1",
    description="Test workflow for inf and eval",
    model="openai/gpt-4o-mini",
    model_url="openai/gpt-4o-mini",
    dataset=str(dataset_id),
    experiment_id=str(experiment_id),
    max_samples=1,
)
workflow_1_response = lumi_client_int.workflows.create_workflow(request)
assert workflow_1_response is not None
workflow_1_id = workflow_1_response.id

In [ ]:
# Wait till the workflow is done
workflow_1_details = wait_for_workflow_complete(lumi_client_int, workflow_1_id)

In [ ]:
# Get the results of the experiment
experiment_results = lumi_client_int.experiments.get_experiment(experiment_id)
assert experiment_results is not None
assert workflow_1_details.experiment_id == experiment_results.id
assert len(experiment_results.workflows) == 1
assert workflow_1_details.model_dump(
    exclude={"artifacts_download_url"}
) == experiment_results.workflows[0].model_dump(exclude={"artifacts_download_url"})

In [ ]:
# Get the results of the experiment
experiment_results = lumi_client_int.experiments.get_experiment(experiment_id)
assert len(experiment_results.workflows) == 2
assert workflow_1_details.model_dump(exclude={"artifacts_download_url"}) in [
    w.model_dump(exclude={"artifacts_download_url"}) for w in experiment_results.workflows
]

# Get the logs
logs_response = lumi_client_int.workflows.get_workflow_logs(workflow_1_details.id)
logs_response

In [ ]:
# Delete the experiment
lumi_client_int.experiments.delete_experiment(experiment_id)
#  shoudl throw exeption: lumi_client_int.experiments.get_experiment(experiment_id)
try:
    lumi_client_int.experiments.get_experiment(experiment_id)
    raise Exception("Should have thrown an exception")
except requests.exceptions.HTTPError:
    pass

try:
    lumi_client_int.workflows.get_workflow(workflow_1_details.id)
    raise Exception("Should have thrown an exception")
except requests.exceptions.HTTPError:
    pass